<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: Convert_to_HydroBase ETKThorn

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we generate the Convert_to_HydroBase ETK thorn files, compatible with our latest implementation of IllinoisGRMHD

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

If using the version of `IllinoisGRMHD` with piecewise polytropic *or* tabulated (coming soon!) EOS support, then the following citation is also required:

* **(Required)** Etienne, Z. B., Werneck, L., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L., *IllinoisGRMHD github repository* (2019). Source Code URL: https://github.com/zachetienne/nrpytutorial/tree/master/IllinoisGRMHD/.

### Dependencies

The files generated in this tutorial notebook depends on the following files:

* `IllinoisGRMHD_headers.h` \[[**tutorial**](Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.ipynb)\]
* `IllinoisGRMHD_EoS_lowlevel_functs.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__EoS_lowlevel_functs.ipynb)\]

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#convert_to_hydrobase__src): **The `Convert_to_HydroBase.C` file**
    1. [Step 1.a](#initialize_eos_struct__compute_eps): *Initializing the EOS struct, computing $\epsilon$*
    1. [Step 1.b](#valencia_vi_from_vi): *Computing the Valencia 3-velocity from $v^{i}$*
    1. [Step 1.c](#computing_lorentz_factor__update_bi): *Computing the Lorentz factor $\gamma_{v}$ and updating $B^{i}$*
1. [Step 2](#convert_to_hydrobase__param): **The `param.ccl` file**
1. [Step 3](#convert_to_hydrobase__interface): **The `interface.ccl` file**
1. [Step 4](#convert_to_hydrobase__schedule): **The `schedule.ccl` file**
1. [Step 5](#convert_to_hydrobase__make): **The `make.code.defn` file**
1. [Step 6](#code_validation): **Code validation**
1. [Step 7](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Load up cmdline_helper and create the directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd
basedir = os.path.join("..","Convert_to_HydroBase")
cmd.mkdir(basedir)
srcdir = os.path.join(basedir,"src")
cmd.mkdir(srcdir)

<a id='convert_to_hydrobase__src'></a>

# Step 1: The `Convert_to_HydroBase.C` file \[Back to [top](#toc)\]
$$\label{convert_to_hydrobase__src}$$

<a id='initialize_eos_struct__compute_eps'></a>

## Step 1.a: Initializing the EOS struct, computing $\epsilon$ \[Back to [top](#toc)\]
$$\label{initialize_eos_struct__compute_eps}$$

Our goal here is to compute $\epsilon$, the specific internal energy. `IllinoisGRMHD` implements a hybrid EOS (see equation (20) in [Etienne *et al*.](https://arxiv.org/pdf/1501.07276.pdf))

$$
P = P_{\rm cold} + P_{\rm th} = P_{\rm cold} + \left(\Gamma_{\rm th} - 1\right)\rho_{b}\epsilon_{\rm th}\ ,
$$

where $P_{\rm cold}$ is determined using a single or piecewise polytropic EOS. We assume $\epsilon = \epsilon_{\rm cold} + \epsilon_{\rm th}$, i.e.

$$
P = P_{\rm cold} + \left(\Gamma_{\rm th} - 1\right)\rho_{b}\left(\epsilon - \epsilon_{\rm cold}\right)\ ,
$$

so that we obtain

$$
\boxed{\epsilon = \epsilon_{\rm cold} + \frac{P - P_{\rm cold}}{\rho_{b}\left(\Gamma_{\rm th}-1\right)}}\ .
$$

Below, $P_{\rm cold}$ and $\epsilon_{\rm cold}$ are determined using the `compute_P_cold__eps_cold()` function, which is documented in the [EoS_lowlevel_functs `IllinoisGRMHD` tutorial notebook](Tutorial-IllinoisGRMHD__EoS_lowlevel_functs.ipynb).

In [2]:
%%writefile $srcdir/Convert_to_HydroBase.C
#include "cctk.h"
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <sys/time.h>
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

#include "IllinoisGRMHD_headers.h"

void Convert_to_HydroBase(CCTK_ARGUMENTS) {

  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // Generally, we only need the HydroBase variables for diagnostic purposes, so we run the below loop only at iterations in which diagnostics are run.
  if(Convert_to_HydroBase_every==0 || cctk_iteration%Convert_to_HydroBase_every!=0) return;

  /***************
   * PPEOS Patch *
   ***************
   * We will need to set up our EOS in
   * order to be able to compute eps below
   */
  eos_struct eos;
  initialize_EOS_struct_from_input(eos);
  
#pragma omp parallel for 
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        int index = CCTK_GFINDEX3D(cctkGH,i,j,k);
        /* Note that we currently do not set Abar, Y_e, temperature, entropy, Avec[3], Aphi, Avec_stag[3], Aphi_stag */
        CCTK_REAL PRIMS[MAXNUMVARS];
        int ww=0;
        PRIMS[ww] = rho_b[index]; ww++;
        PRIMS[ww] = P[index];     ww++;
        PRIMS[ww] = vx[index];    ww++;
        PRIMS[ww] = vy[index];    ww++;
        PRIMS[ww] = vz[index];    ww++;
        PRIMS[ww] = Bx[index];    ww++;
        PRIMS[ww] = By[index];    ww++;
        PRIMS[ww] = Bz[index];    ww++;

        rho[index]   = PRIMS[RHOB];
        press[index] = PRIMS[PRESSURE];

        /***************
         * PPEOS Patch *
         ***************
         * For our hybrid piecewise polytropic EOS, we have
         * .------------------------------------------------------.
         * | eps = eps_cold + (P - P_cold)/( rho*(Gamma_th - 1) ) |
         * .------------------------------------------------------.
         */
        /* Compute P_cold and eps_cold */
        CCTK_REAL P_cold, eps_cold;
        compute_P_cold__eps_cold(eos,PRIMS[RHOB], P_cold,eps_cold);

        /* Compute eps as described above */
        eps[index] = (PRIMS[PRESSURE]-P_cold)/PRIMS[RHOB]/(Gamma_th-1.0);

Overwriting ../Convert_to_HydroBase/src/Convert_to_HydroBase.C


<a id='valencia_vi_from_vi'></a>

## Step 1.b: Computing the Valencia 3-velocity from $v^{i}$ \[Back to [top](#toc)\]
$$\label{valencia_vi_from_vi}$$

In `IllinoisGRMHD`, we use the 3-velocity

$$
v^{i}\equiv\frac{u^{i}}{u^{0}}\ ,
$$

where $u^{\mu}$ is the fluid 4-velocity. The [HydroBase ETK Thorn](http://einsteintoolkit.org/thornguide/EinsteinBase/HydroBase/documentation.html), however, uses the Valencia formalism, which adopts the 3-velocity as measured by a normal observers (also referred to as the Eulerian 3-velocity), $v^{i}_{(n)}$, which is given by (see equation (14) in [Mösta *et al*.](https://arxiv.org/pdf/1304.5544.pdf))

$$
v^{i}_{(n)} = \frac{u^{i}}{\alpha u^{0}} + \frac{\beta^{i}}{\alpha}\ .
$$

Thus, we can read off the relation

$$
\boxed{v^{i}_{(n)} = \frac{1}{\alpha}\left(v^{i} + \beta^{i}\right)}\ ,
$$

which we implement below.

In [3]:
%%writefile -a $srcdir/Convert_to_HydroBase.C


        // IllinoisGRMHD defines v^i = u^i/u^0.
        
        // Meanwhile, the ET/HydroBase formalism, called the Valencia 
        // formalism, splits the 4 velocity into a purely spatial part
        // and a part that is normal to the spatial hypersurface:
        // u^a = G (n^a + U^a), (Eq. 14 of arXiv:1304.5544; G=W, U^a=v^a)
        // where n^a is the unit normal vector to the spatial hypersurface,
        // n_a = {-\alpha,0,0,0}, and U^a is the purely spatial part, which
        // is defined in HydroBase as the vel[] vector gridfunction.
        // Then u^a n_a = - \alpha u^0 = G n^a n_a = -G, and
        // of course \alpha u^0 = 1/sqrt(1+γ^ij u_i u_j) = \Gamma,
        // the standard Lorentz factor.

        // Note that n^i = - \beta^i / \alpha, so 
        // u^a = \Gamma (n^a + U^a) 
        // -> u^i = \Gamma ( U^i - \beta^i / \alpha )
        // which implies
        // v^i = u^i/u^0
        //     = \Gamma/u^0 ( U^i - \beta^i / \alpha ) <- \Gamma = \alpha u^0
        //     = \alpha ( U^i - \beta^i / \alpha )
        //     = \alpha U^i - \beta^i
        CCTK_REAL lapseL=alp[index];
        CCTK_REAL lapseL_inv=1.0/lapseL;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,0)] = (PRIMS[VX] + betax[index])*lapseL_inv;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,1)] = (PRIMS[VY] + betay[index])*lapseL_inv;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,2)] = (PRIMS[VZ] + betaz[index])*lapseL_inv;

Appending to ../Convert_to_HydroBase/src/Convert_to_HydroBase.C


<a id='computing_lorentz_factor__update_bi'></a>

## Step 1.c: Computing the Lorentz factor $\gamma_{v}$ and updating $B^{i}$ \[Back to [top](#toc)\]
$$\label{computing_lorentz_factor__update_bi}$$

We now want to compute the Lorentz factor $\gamma_{v}=\left(1+\gamma_{ij}u^{i}u^{j}\right)^{-1/2}$. Below we implement the quantity

$$
A = 1 - \frac{1}{\left(\alpha u^{0}\right)^{2}} = \gamma_{ij}\left(\frac{v^{i}+\beta^{i}}{\alpha}\right)\left(\frac{v^{j}+\beta^{j}}{\alpha}\right)\ ,
$$

which imply that $\gamma_{v}=\alpha u^{0}$. First let us show that this is indeed the case. Consider

$$
\begin{align}
A 
&= \gamma_{ij}\left(\frac{v^{i}+\beta^{i}}{\alpha}\right)\left(\frac{v^{j}+\beta^{j}}{\alpha}\right)\\
&= \frac{\gamma_{ij}}{\alpha^{2}}\left[\frac{\gamma^{ik}u_{k}}{u^{0}} - \beta^{i} + \beta^{i}\right]\left[\frac{\gamma^{j\ell}u_{\ell}}{u^{0}} - \beta^{j} + \beta^{j}\right]\\
&=\frac{\gamma_{ij}u^{i}u^{j}}{\left(\alpha u^{0}\right)^{2}}\\
&=\frac{\left(\alpha u^{0}\right)^{2}-1}{\left(\alpha u^{0}\right)^{2}}\\
&=1 - \frac{1}{\left(\alpha u^{0}\right)^{2}}\ \\
\implies &\boxed{A = 1 - \frac{1}{\left(\alpha u^{0}\right)^{2}}}\ ,
\end{align}
$$

where when going from line 1 to 2 and from line 3 to 4 we have used eqs. (53) and (56) from [Duez *et al.*](https://arxiv.org/pdf/astro-ph/0503420.pdf), respectively. Keep in mind that the equation we are going to implement below is

$$
\boxed{{\rm one\_minus\_one\_over\_alpha\_u0\_squared} = \gamma_{ij}\left(\frac{v^{i}+\beta^{i}}{\alpha}\right)\left(\frac{v^{j}+\beta^{j}}{\alpha}\right)}\ ,
$$

but it is important to know that this equation also equals $A$ above.

Finally, we update the `Bvec` array with the appropriate (`Bx`,`By`,`Bz`) values.

In [4]:
%%writefile -a $srcdir/Convert_to_HydroBase.C


        // \alpha u^0 = 1/sqrt(1+γ^ij u_i u_j) = \Gamma = w_lorentz
        // First compute u^0:
        // Derivation of first equation:
        // \gamma_{ij} (v^i + \beta^i)(v^j + \beta^j)/(\alpha)^2 
        //   = \gamma_{ij} 1/(u^0)^2 ( \gamma^{ik} u_k \gamma^{jl} u_l /(\alpha)^2 <- Using Eq. 53 of arXiv:astro-ph/0503420
        //   = 1/(u^0 \alpha)^2 u_j u_l \gamma^{jl}  <- Since \gamma_{ij} \gamma^{ik} = \delta^k_j
        //   = 1/(u^0 \alpha)^2 ( (u^0 \alpha)^2 - 1 ) <- Using Eq. 56 of arXiv:astro-ph/0503420
        //   = 1 - 1/(u^0 \alpha)^2 <= 1
        CCTK_REAL shiftxL = betax[index];
        CCTK_REAL shiftyL = betay[index];
        CCTK_REAL shiftzL = betaz[index];

        CCTK_REAL gxxL = gxx[index];
        CCTK_REAL gxyL = gxy[index];
        CCTK_REAL gxzL = gxz[index];
        CCTK_REAL gyyL = gyy[index];
        CCTK_REAL gyzL = gyz[index];
        CCTK_REAL gzzL = gzz[index];

        CCTK_REAL one_minus_one_over_alpha_u0_squared = (gxxL* SQR(PRIMS[VX] + shiftxL) +
                                                         2.0*gxyL*(PRIMS[VX] + shiftxL)*(PRIMS[VY] + shiftyL) +
                                                         2.0*gxzL*(PRIMS[VX] + shiftxL)*(PRIMS[VZ] + shiftzL) +
                                                         gyyL* SQR(PRIMS[VY] + shiftyL) +
                                                         2.0*gyzL*(PRIMS[VY] + shiftyL)*(PRIMS[VZ] + shiftzL) +
                                                         gzzL* SQR(PRIMS[VZ] + shiftzL) )*SQR(lapseL_inv);
        /*** Check for superluminal velocity ***/
        //FIXME: Instead of >1.0, should be one_minus_one_over_alpha_u0_squared > ONE_MINUS_ONE_OVER_GAMMA_SPEED_LIMIT_SQUARED, for consistency with conserv_to_prims routines

        if(one_minus_one_over_alpha_u0_squared > 1.0) {
          CCTK_VInfo(CCTK_THORNSTRING,"Convert_to_HydroBase WARNING: Found superluminal velocity. This should have been caught by IllinoisGRMHD.");
        }

        // A = 1.0-one_minus_one_over_alpha_u0_squared = 1-(1-1/(al u0)^2) = 1/(al u0)^2
        // 1/sqrt(A) = al u0
        CCTK_REAL alpha_u0 = 1.0/sqrt(1.0-one_minus_one_over_alpha_u0_squared);
        if(std::isnan(alpha_u0*lapseL_inv)) printf("BAD FOUND NAN ALPHAU0 CALC: %.15e %.15e %.15e\n",alpha_u0,lapseL_inv,one_minus_one_over_alpha_u0_squared);

        w_lorentz[index] = alpha_u0;

        Bvec[CCTK_GFINDEX4D(cctkGH,i,j,k,0)] = PRIMS[BX_CENTER];
        Bvec[CCTK_GFINDEX4D(cctkGH,i,j,k,1)] = PRIMS[BY_CENTER];
        Bvec[CCTK_GFINDEX4D(cctkGH,i,j,k,2)] = PRIMS[BZ_CENTER];

      }
}


Appending to ../Convert_to_HydroBase/src/Convert_to_HydroBase.C


<a id='convert_to_hydrobase__param'></a>

# Step 2: The `param.ccl` file \[Back to [top](#toc)\]
$$\label{convert_to_hydrobase__param}$$

The `param.ccl` file sets up parameters that are used by the `Convert_to_HydroBase` ETK thorn. These parameters are accessed as globals after calling the `DECLARE_CCTK_PARAMETERS` macro.

When declaring a parameter, we have the following notation

<pre>
<font color="blue">TYPE</font> <font color="green">name</font> <font color="red">"Comment"</font> <font color="purple">STEERABLE=?</font>
{
minimum allowed value:maximum allowed value :: <font color="red">"Comment"</font>
other possible value #1                     :: <font color="red">"Comment"</font>
other possible value #2                     :: <font color="red">"Comment"</font>
         ...                                ::    ...
final possible value                        :: <font color="red">"Comment"</font>
} <font color="orange">Default value</font>
</pre>


The *STEERABLE* option is either set to never (default value, for which the keyword is not necessary at all) or to ALWAYS, which allows for the value of the parameter to change after set to its initial value. For example, when declaring the $\Gamma_{\rm th}$ parameter below, we have

<pre>
<font color="blue">REAL</font> <font color="green">Gamma_th</font> <font color="red">"thermal gamma parameter"</font>
{
 0:* :: <font color="red">"Physical values"</font>
-1   :: <font color="red">"forbidden value to make sure it is explicitly set in the parfile"</font>
} <font color="orange">-1</font>
</pre>


This declares the *REAL* (usually a macro for double) $\Gamma_{\rm th}$ parameter as ${\rm Gamma\_th}$. We allow this parameter to have only positive values, hence the notation `0:*`. The asterisk means "any positive value is allowed". The notation `*:*` then means "any value is allowed. The second line above sets the value "$-1$" to be a forbidden value, which if read after the code starts would produce an error saying that this value has not been initialized properly. Finally, the default value of the variable is set to "$-1$", the forbidden value. Because the STEERABLE keyword is not explicitly set to ALWAYS above, this means that once set, the value of $\Gamma_{\rm th}$ cannot be changed.

The `shares: THORN` syntax tells the ETK that the `Convert_to_HydroBase` thorn will share the parameters from the `THORN` thorn. For example, the following code

<pre>
<font color="purple">shares:</font> <font color="blue">ADMBase</font>
<font color="purple">USES CCTK_INT</font> <font color="green">lapse_timelevels</font> 
<font color="purple">USES CCTK_INT</font> <font color="green">shift_timelevels</font> 
<font color="purple">USES CCTK_INT</font> <font color="green">metric_timelevels</font> 
</pre>

will tell the ETK that the variables `lapse_timelevels`, `shift_timelevels`, and `metric_timelevels` from the [ADMBase ETK thorn](http://einsteintoolkit.org/thornguide/EinsteinBase/ADMBase/documentation.html) should also be available for the `Convert_to_HydroBase` thorn once the `DECLARE_CCTK_PARAMETERS` macro has been executed.

For more details on how to build a `param.ccl` file, please refer to the [official Cactus documentation](https://einsteintoolkit.org/usersguide/UsersGuidech12.html).

In [5]:
%%writefile $basedir/param.ccl
# Parameter definitions for thorn convert_to_HydroBase
# $Header:$

#############################################################################
### import HydroBase & ADMBase parameters

shares: HydroBase
USES CCTK_INT timelevels

shares: ADMBase
USES CCTK_INT lapse_timelevels
USES CCTK_INT shift_timelevels
USES CCTK_INT metric_timelevels

shares: IllinoisGRMHD
USES KEYWORD neos
USES KEYWORD Gamma_th
USES KEYWORD K_ppoly_tab0
USES KEYWORD rho_ppoly_tab_in[10]
USES KEYWORD Gamma_ppoly_tab_in[10]
#############################################################################

private:
INT Convert_to_HydroBase_every "How often to convert IllinoisGRMHD primitive variables to HydroBase (Valencia formulation) primitive variables? Needed for some ET-based diagnostics. NOT needed for pure IllinoisGRMHD runs."
{
 0:* :: "zero (disable) or positive (every N iterations)"
} 0



Overwriting ../Convert_to_HydroBase/param.ccl


<a id='convert_to_hydrobase__interface'></a>

# Step 3: The `interface.ccl` file \[Back to [top](#toc)\]
$$\label{convert_to_hydrobase__interface}$$


The `interface.ccl` file sets up a few different things of how `Convert_to_HydroBase` interfaces with the rest of the toolkit. For starters, it tells which thorns are used by `Convert_to_HydroBase`, which is done using the keyword `inherits`.

It also requests to use files from different thorns using the `uses include header: HEADER_FILE.h` command.

For more details on how to build a `interface.ccl` file, please refer to the [official Cactus documentation](https://einsteintoolkit.org/usersguide/UsersGuidech12.html).

In [6]:
%%writefile $basedir/interface.ccl
# Interface definition for thorn Convert_to_HydroBase
# $Header:$

implements: Convert_to_HydroBase
inherits: grid HydroBase ADMBase IllinoisGRMHD

uses include header: IllinoisGRMHD_headers.h



Overwriting ../Convert_to_HydroBase/interface.ccl


<a id='convert_to_hydrobase__schedule'></a>

# Step 4: The `schedule.ccl` file \[Back to [top](#toc)\]
$$\label{convert_to_hydrobase__schedule}$$

The `schedule.ccl` file sets up the structure of a given ETK run. It tells when the functions of `Convert_to_HydroBase` must be called and in what order. The general form of a scheduling is

<pre>
<font color="blue">schedule</font> <font color="green">name</font> <font color="purple">IN</font> <font color="orange">thorn_name</font> <font color="purple">as</font> <font color="green">alias</font> <font color="purple">after</font> <font color="green">afunct_1</font> ... <font color="purple">after</font> <font color="green">afunct_n</font> <font color="purple">before</font> <font color="green">bfunct_1</font> ... <font color="purple">before</font> <font color="green">bfunct_m</font>
{
LANG:    <font color="purple">function programming language</font>
OPTIONS: <font color="purple">options</font>
SYNC:    <font color="purple">sync options</font>
} <font color="red">"Comment"</font>
</pre>

Following the notation above, the <font color="blue">schedule</font> keyword sets up the function whose name is <font color="green">name</font> defined <font color="purple">IN</font> the ETK thorn <font color="orange">thorn_name</font>. We are also allowed to set an <font color="green">alias</font>, if the function name is too long, but this is optional. We then set exactly when the function should be called, i.e. by saying it should be called <font color="purple">before</font> and/or <font color="purple">after</font> other functions.

Inside curly braces, we set up the programming language of the function, e.g. C, options, e.g. LOCAL, and whether or not variables need to be synchronized with other functions. Finally, one can place a commend outside the curly braces, explaining the schedule just implemented.

For more details on how to build a `schedule.ccl` file, please refer to the [official Cactus documentation](https://einsteintoolkit.org/usersguide/UsersGuidech12.html).

In [7]:
%%writefile $basedir/schedule.ccl
# Schedule definitions for thorn Convert_to_HydroBase
# $Header:$

SCHEDULE Convert_to_HydroBase AT CCTK_INITIAL AFTER SetTmunu
{
  LANG: C
} "Convert IllinoisGRMHD-native variables to HydroBase"

SCHEDULE Convert_to_HydroBase AT CCTK_ANALYSIS BEFORE compute_bi_b2_Poyn_fluxET BEFORE particle_tracerET BEFORE VolumeIntegralGroup BEFORE convert_to_MHD_3velocity AFTER ML_BSSN_evolCalcGroup
{
  OPTIONS: GLOBAL-EARLY,LOOP-LOCAL
  LANG: C
} "Convert IllinoisGRMHD-native variables to HydroBase"



Overwriting ../Convert_to_HydroBase/schedule.ccl


<a id='convert_to_hydrobase__make'></a>

# Step 5: The `make.code.defn` file \[Back to [top](#toc)\]
$$\label{convert_to_hydrobase__make}$$

The `make.code.defn` file tells the toolkit which files should be compiled during the ETK build.

In [8]:
%%writefile $srcdir/make.code.defn
# Main make.code.defn file for thorn Convert_to_HydroBase
# $Header:$

# Source files in this directory
SRCS = Convert_to_HydroBase.C

# Subdirectories containing source files
SUBDIRS = 



Overwriting ../Convert_to_HydroBase/src/make.code.defn


<a id='code_validation'></a>

# Step 6: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [9]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/A_i_rhs_no_gauge_terms.C"
# original_IGM_file_name = "A_i_rhs_no_gauge_terms-original.C"
# original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__A_i_rhs_no_gauge_terms__C  = !diff $original_IGM_file_path $outfile_path__A_i_rhs_no_gauge_terms__C

# if Validation__A_i_rhs_no_gauge_terms__C == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for A_i_rhs_no_gauge_terms.C: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for A_i_rhs_no_gauge_terms.C: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__A_i_rhs_no_gauge_terms__C:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.pdf](Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [10]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__ETKThorn__Convert_to_HydroBase.tex
!rm -f Tut*.out Tut*.aux Tut*.log